# Predict Sales

> Created on 04/04/2021, 2:02 PM IST

## Data Preparation

1. Checking percentage of NA values
2. Dropping NA values 
3. Imputing NA values
4. Renaming Column Names
5. Converting Data Types to datetime64[ns] format.

In [1]:
import pandas as pd
import numpy as np 
import sklearn 
import matplotlib.pyplot as plt
import plotly
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder


In [2]:
df = pd.read_csv("predict_revenue.csv")
print(df.shape)
df.describe()

(270, 24)


,order_no,discount_amount,adults,booking_fee,paid_from_wallet,commission,bf_discount,trip_id,net_revenue,base_id,ff_discount,facility_fee,amount_in_usd,voucher_consumption,paid_from_card
count,270.000000,270.000000,270.000000,270.0,270.0,270.0,270.0,270.000000,270.000000,270.000000,270.0,270.0,270.000000,270.0,270.000000
mean,776461.233333,9.316667,1.003704,0.0,0.0,0.0,0.0,172440.592593,1.782407,78203.188889,0.0,0.0,11.099074,0.0,1.782407
std,2237.984336,4.101234,0.060858,0.0,0.0,0.0,0.0,120170.540314,3.999287,97252.981282,0.0,0.0,0.800586,0.0,3.999287
min,774016.000000,0.000000,1.000000,0.0,0.0,0.0,0.0,16009.000000,0.000000,35.000000,0.0,0.0,10.250000,0.0,0.000000
25%,774936.750000,10.250000,1.000000,0.0,0.0,0.0,0.0,16009.000000,0.000000,251.750000,0.0,0.0,11.250000,0.0,0.000000
50%,775962.500000,11.250000,1.000000,0.0,0.0,0.0,0.0,168171.000000,0.000000,19819.000000,0.0,0.0,11.250000,0.0,0.000000
75%,777875.250000,11.250000,1.000000,0.0,0.0,0.0,0.0,310958.000000,0.000000,138113.000000,0.0,0.0,11.250000,0.0,0.000000
max,782807.000000,22.500000,2.000000,0.0,0.0,0.0,0.0,310958.000000,11.250000,319870.000000,0.0,0.0,22.500000,0.0,11.250000


In [3]:
df.head()

,order_no,discount_amount,adults,booking_fee,paid_from_wallet,order_time,source,isRelease,trip_type,leg_name,...,isCancel,net_revenue,base_id,travel_date,ff_discount,facility_fee,amount_in_usd,voucher_consumption,paid_from_card,pick_stop
0,774016,11.25,1,0,0,09-25-2019 3:47:29,WEB,False,C,Kendall Park to NYC Commuter,...,False,0.0,294729,09-25-2019,0,0,11.25,0,0.0,Kendall Park Roller Skating Rink
1,774017,11.25,1,0,0,09-25-2019 3:48:52,WEB,False,C,"New York, NY - Kendall Park Commuter",...,False,0.0,294729,09-25-2019,0,0,11.25,0,0.0,Times Square
2,774027,10.25,1,0,0,09-25-2019 5:24:09,IOS,False,C,Kendall Park to NYC Commuter,...,False,0.0,1338,09-25-2019,0,0,10.25,0,0.0,Kendall Park Roller Skating Rink
3,774027,10.25,1,0,0,09-25-2019 5:24:09,IOS,False,C,"New York, NY - Kendall Park Commuter",...,False,0.0,1338,09-25-2019,0,0,10.25,0,0.0,Kendall Park Roller Skating Rink
4,774029,11.25,1,0,0,09-25-2019 5:38:44,WEB,False,C,Kendall Park to NYC Commuter,...,False,0.0,35,09-25-2019,0,0,11.25,0,0.0,Kendall Park Post Office


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 270 entries, 0 to 269
Data columns (total 24 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   order_no             270 non-null    int64  
 1   discount_amount      270 non-null    float64
 2   adults               270 non-null    int64  
 3   booking_fee          270 non-null    int64  
 4   paid_from_wallet     270 non-null    int64  
 5   order_time           270 non-null    object 
 6   source               270 non-null    object 
 7   isRelease            270 non-null    bool   
 8   trip_type            270 non-null    object 
 9   leg_name             270 non-null    object 
 10  commission           270 non-null    int64  
 11  departure_time       270 non-null    object 
 12  bf_discount          270 non-null    int64  
 13  trip_id              270 non-null    int64  
 14  isCancel             270 non-null    bool   
 15  net_revenue          270 non-null    flo

In [5]:

## Percentage of missing values in each column

missing_values = pd.DataFrame({c:(df[c].isna().sum()/len(df))*100 for c in df.columns},index=["% missing values in df"])

# Drop NA if < 5% (if any)
df  =  df.dropna()
print(df.shape)

# Drop Duplicates 
df = df.drop_duplicates()
print(df.shape)

(270, 24)
(270, 24)


The only date-time columns available here are order_time and travel_date

In [6]:
df['order_time'] = pd.to_datetime(df['order_time'])
df['travel_date'] = pd.to_datetime(df['travel_date'])
df['departure_time'] = pd.to_datetime(df['departure_time'])

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 270 entries, 0 to 269
Data columns (total 24 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   order_no             270 non-null    int64         
 1   discount_amount      270 non-null    float64       
 2   adults               270 non-null    int64         
 3   booking_fee          270 non-null    int64         
 4   paid_from_wallet     270 non-null    int64         
 5   order_time           270 non-null    datetime64[ns]
 6   source               270 non-null    object        
 7   isRelease            270 non-null    bool          
 8   trip_type            270 non-null    object        
 9   leg_name             270 non-null    object        
 10  commission           270 non-null    int64         
 11  departure_time       270 non-null    datetime64[ns]
 12  bf_discount          270 non-null    int64         
 13  trip_id              270 non-null  

In [8]:
df['order_day'] = pd.DatetimeIndex(df['order_time']).day
df['order_hour'] = pd.DatetimeIndex(df['order_time']).hour

# Since the year belongs to only 2019 and seconds won't matter
# Since there are only 4 days on which the travel has happened

df['departure_hour'] = pd.DatetimeIndex(df['departure_time']).hour
df['departure_minute'] = pd.DatetimeIndex(df['departure_time']).minute

df['travel_day'] = pd.DatetimeIndex(df['travel_date']).day



In [9]:
# Dropping the columns which contain only a single value because a single value in all the rows won't have any affect on the prediction 
redundant_columns = ['voucher_consumption', 'facility_fee', 'facility_fee', 'ff_discount', 'isCancel', 'bf_discount', 'commission', 'trip_type', 'isRelease', 'order_time', 'travel_date', 'paid_from_wallet', 'booking_fee' , 'departure_time', 'order_no']

# Adding order_no in the redundant columns because it should not be a dependent variable in deciding the revenue.

categorical_columns = ['pick_stop', 'trip_id', 'leg_name', 'source', ]

df = df.drop(columns = redundant_columns,)
df

,discount_amount,adults,source,leg_name,trip_id,net_revenue,base_id,amount_in_usd,paid_from_card,pick_stop,order_day,order_hour,departure_hour,departure_minute,travel_day
0,11.25,1,WEB,Kendall Park to NYC Commuter,16009,0.0,294729,11.25,0.0,Kendall Park Roller Skating Rink,25,3,6,25,25
1,11.25,1,WEB,"New York, NY - Kendall Park Commuter",310958,0.0,294729,11.25,0.0,Times Square,25,3,17,30,25
2,10.25,1,IOS,Kendall Park to NYC Commuter,16009,0.0,1338,10.25,0.0,Kendall Park Roller Skating Rink,25,5,6,25,25
3,10.25,1,IOS,"New York, NY - Kendall Park Commuter",310958,0.0,1338,10.25,0.0,Kendall Park Roller Skating Rink,25,5,17,30,25
4,11.25,1,WEB,Kendall Park to NYC Commuter,16009,0.0,35,11.25,0.0,Kendall Park Post Office,25,5,6,25,25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
265,11.25,1,IOS,Kendall Park to NYC Commuter,117711,0.0,430,11.25,0.0,Kendall Park Post Office,29,21,6,55,30
266,11.25,1,WEB,Kendall Park to NYC Commuter,16009,0.0,294729,11.25,0.0,Kendall Park Roller Skating Rink,29,22,6,25,30
267,11.25,1,WEB,"New York, NY - Kendall Park Commuter",310958,0.0,294729,11.25,0.0,Times Square,29,22,17,30,30
268,11.25,1,IOS,Kendall Park to NYC Commuter,117711,0.0,260,11.25,0.0,Kendall Park Roller Skating Rink,29,22,6,55,30


In [10]:
# One Hot encoding 
ohe = OneHotEncoder(categories='auto')
feature_arr = ohe.fit_transform(df[categorical_columns]).toarray()
feature_labels = ohe.categories_

In [11]:
feature_df = pd.DataFrame(feature_arr)

try:
    df = df.drop(columns = categorical_columns)
except:
    pass

# concatenating feature_df with existing df

main_df = pd.concat([df, feature_df], axis=1)

main_df

,discount_amount,adults,net_revenue,base_id,amount_in_usd,paid_from_card,order_day,order_hour,departure_hour,departure_minute,...,5,6,7,8,9,10,11,12,13,14
0,11.25,1,0.0,294729,11.25,0.0,25,3,6,25,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
1,11.25,1,0.0,294729,11.25,0.0,25,3,17,30,...,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
2,10.25,1,0.0,1338,10.25,0.0,25,5,6,25,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
3,10.25,1,0.0,1338,10.25,0.0,25,5,17,30,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
4,11.25,1,0.0,35,11.25,0.0,25,5,6,25,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
265,11.25,1,0.0,430,11.25,0.0,29,21,6,55,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
266,11.25,1,0.0,294729,11.25,0.0,29,22,6,25,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
267,11.25,1,0.0,294729,11.25,0.0,29,22,17,30,...,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
268,11.25,1,0.0,260,11.25,0.0,29,22,6,55,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0


## Train Test Split 

In [12]:
main_df['target'] = main_df['adults'] * main_df['amount_in_usd']

X = main_df.drop(['target'], axis = 1)
Y = main_df['target']

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.10,random_state=0)

print(X_train.shape, X_test.shape, Y_train.shape, Y_test.shape)

(243, 26) (27, 26) (243,) (27,)


## Model 

1. Decision Tree Regressor
2. Random Forest Regressor

In [13]:
dt_reg = DecisionTreeRegressor(random_state=0)
dt_reg.fit(X_train, Y_train)

DecisionTreeRegressor(random_state=0)

In [14]:
# Feature Importance
# sum adult group by date 
def feature_imp(model, X_test, Y_test):
    print("Feature Importance:\n")
    for name, importance in zip(X.columns, np.sort(model.feature_importances_)[::-1]):print("{} --{:.2f}".format(name, importance))


    print("score" , model.score(X_test, Y_test))
    y_pred = model.predict(X_test)
    return y_pred


In [15]:
feature_imp(dt_reg, X_test, Y_test)


Feature Importance:

discount_amount --0.97
adults --0.03
net_revenue --0.00
base_id --0.00
amount_in_usd --0.00
paid_from_card --0.00
order_day --0.00
order_hour --0.00
departure_hour --0.00
departure_minute --0.00
travel_day --0.00
0 --0.00
1 --0.00
2 --0.00
3 --0.00
4 --0.00
5 --0.00
6 --0.00
7 --0.00
8 --0.00
9 --0.00
10 --0.00
11 --0.00
12 --0.00
13 --0.00
14 --0.00
score 1.0


array([11.25, 11.25, 11.25, 10.25, 10.25, 11.25, 11.25, 11.25, 10.25,
       11.25, 11.25, 11.25, 11.25, 10.25, 11.25, 11.25, 11.25, 10.25,
       10.25, 11.25, 11.25, 11.25, 11.25, 11.25, 10.25, 11.25, 11.25])

In [17]:
rf_reg = RandomForestRegressor(random_state=0)
rf_reg.fit(X_train, Y_train)
feature_imp(rf_reg, X_test, Y_test)

RandomForestRegressor(random_state=0)

In [21]:
predict_df = pd.DataFrame()
predict_df['DT'] = dt_reg.predict(X_test)
predict_df['RF'] = rf_reg.predict(X_test)
predict_df

,DT,RF
0,11.25,11.25
1,11.25,11.25
2,11.25,11.25
3,10.25,10.25
4,10.25,10.25
5,11.25,11.25
6,11.25,11.25
7,11.25,11.25
8,10.25,10.25
9,11.25,11.25


In [25]:
svr = sklearn.svm.SVR
svr.fit(X = X_train,y = Y_train)

TypeError: fit() missing 1 required positional argument: 'self'